### ***`Its a spacial runnable primitive that simply` returns the input as the output without modifying it***
* its also help to get intermediate chain output

In [3]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
import os
from dotenv import load_dotenv
load_dotenv()
from langchain_groq import ChatGroq
from langchain_anthropic import ChatAnthropic


groq_api_key=os.getenv("GROQ_API_KEY")

model= ChatGroq(groq_api_key=groq_api_key,model_name="qwen-2.5-32b")

### Explore RunnablePassTrough

In [2]:
from langchain.schema.runnable import RunnablePassthrough, RunnableSequence, RunnableParallel

In [4]:
passthrough = RunnablePassthrough()

In [5]:
passthrough.invoke(2)

2

In [6]:
passthrough.invoke(
    {
        'name': 'Al Amin',
        'age': 25
    }
)

{'name': 'Al Amin', 'age': 25}

## ***Main Code***

In [9]:
prompt = PromptTemplate(
    template="Write a joke about this {topic}",
    input_variables=['topic']
)

prompt2 = PromptTemplate(
    template="Explain the following joke {text}",
    input_variables=['text']
)

In [8]:
parser = StrOutputParser()

In [10]:
joke_gen_chain = RunnableSequence(prompt, model, parser)

In [11]:
parallel_chain = RunnableParallel(
    {
        'joke': RunnablePassthrough(),
        'explanation': RunnableSequence(prompt2, model, parser)
    }
)

In [12]:
final_chain = RunnableSequence(joke_gen_chain,parallel_chain)

In [13]:
final_chain.get_graph().print_ascii()

                 +-------------+                 
                 | PromptInput |                 
                 +-------------+                 
                        *                        
                        *                        
                        *                        
                +----------------+               
                | PromptTemplate |               
                +----------------+               
                        *                        
                        *                        
                        *                        
                  +----------+                   
                  | ChatGroq |                   
                  +----------+                   
                        *                        
                        *                        
                        *                        
               +-----------------+               
               | StrOutputParser |               


In [14]:
response = final_chain.invoke(
    {
        'topic': 'Computer'
    }
)

In [15]:
response['joke']

"Why don't computers ever get lost?\n\nBecause they always know their bytes from their elbows!"

In [16]:
response['explanation']

'This joke plays on the similarity in sound between "bytes" and "elbows" and the inherent nature of computers. In computing, a "byte" is a unit of digital information, typically made up of 8 bits. It\'s a fundamental term in the language of computers. On the other hand, "elbows" are body parts and have nothing to do with computing or navigation.\n\nThe joke suggests that computers wouldn\'t get lost because they can distinguish between their data (bytes) and their physical body parts (elbows), which is a humorous and nonsensical concept since computers don\'t have physical parts like elbows. This contrast creates the humor because it mixes the digital world with the physical in an unexpected way.'